# **1. Importing the Libraries**

In [112]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgbm
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

# **2. Loading the Data**

In [113]:
train = pd.read_csv('/content/drive/MyDrive/Competitions/IndabaX Ghana AutoInland Vehicle Insurance Claim/IndabaX_Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Competitions/IndabaX Ghana AutoInland Vehicle Insurance Claim/IndabaX_Test.csv')
SS = pd.read_csv('/content/drive/MyDrive/Competitions/IndabaX Ghana AutoInland Vehicle Insurance Claim/SampleSubmission.csv')
StateName = pd.read_csv('/content/drive/MyDrive/Competitions/IndabaX Ghana AutoInland Vehicle Insurance Claim/NigerianStateNames.csv')

In [114]:
train.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1


# **3. Exploring and Transforming Data**

In [115]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      1202 non-null   object
 1   Policy Start Date       1202 non-null   object
 2   Policy End Date         1202 non-null   object
 3   Gender                  1161 non-null   object
 4   Age                     1202 non-null   int64 
 5   First Transaction Date  1202 non-null   object
 6   No_Pol                  1202 non-null   int64 
 7   Car_Category            830 non-null    object
 8   Subject_Car_Colour      505 non-null    object
 9   Subject_Car_Make        954 non-null    object
 10  LGA_Name                546 non-null    object
 11  State                   546 non-null    object
 12  ProductName             1202 non-null   object
dtypes: int64(2), object(11)
memory usage: 122.2+ KB


In [116]:
# Combine train and test set
ntrain = train.shape[0] # to be used to split train and test set from the combined dataframe

all_data = pd.concat((train, test)).reset_index(drop=True)

In [117]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      13281 non-null  object 
 1   Policy Start Date       13281 non-null  object 
 2   Policy End Date         13281 non-null  object 
 3   Gender                  12881 non-null  object 
 4   Age                     13281 non-null  int64  
 5   First Transaction Date  13281 non-null  object 
 6   No_Pol                  13281 non-null  int64  
 7   Car_Category            9171 non-null   object 
 8   Subject_Car_Colour      5622 non-null   object 
 9   Subject_Car_Make        10557 non-null  object 
 10  LGA_Name                6149 non-null   object 
 11  State                   6137 non-null   object 
 12  ProductName             13281 non-null  object 
 13  target                  12079 non-null  float64
dtypes: float64(1), int64(2), object(11)
me

In [118]:
for col in all_data.columns:
  print(f'The number of unique values of {col}: {len(train[col].unique())}')

The number of unique values of ID: 12079
The number of unique values of Policy Start Date: 376
The number of unique values of Policy End Date: 372
The number of unique values of Gender: 8
The number of unique values of Age: 110
The number of unique values of First Transaction Date: 376
The number of unique values of No_Pol: 8
The number of unique values of Car_Category: 17
The number of unique values of Subject_Car_Colour: 46
The number of unique values of Subject_Car_Make: 75
The number of unique values of LGA_Name: 259
The number of unique values of State: 112
The number of unique values of ProductName: 9
The number of unique values of target: 2


In [119]:
all_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,13281.0,42.375574,93.170951,-6099.0,35.0,41.0,50.0,320.0
No_Pol,13281.0,1.302763,0.723195,1.0,1.0,1.0,1.0,10.0
target,12079.0,0.120457,0.325509,0.0,0.0,0.0,0.0,1.0


## **3.1. Dealing With Age column**

In [120]:
all_data[all_data['Age']<0]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
909,ID_2SKRJKI,2010-04-30,2011-04-29,Female,-76,2010-04-30,2,NaN,White,Hyundai,Ilupeju,Lagos,CVTP,0.0
1226,ID_3QOZW9J,2010-03-15,2011-03-14,Male,-2,2010-03-15,1,Saloon,Black,TOYOTA,Ibadan South East,Ibadan-East,Customized Motor,0.0
2580,ID_7OMERTL,2010-04-23,2011-04-22,Male,-12,2010-04-23,3,Saloon,As Attached,Hyundai,Victoria Island,Lagos,Car Plus,0.0
3114,ID_98EVCPR,2010-06-26,2011-06-25,Female,-2,2010-06-26,1,JEEP,NaN,TOYOTA,NaN,NaN,Car Classic,0.0
3427,ID_A3YN75Z,2010-10-02,2011-10-01,NOT STATED,-6099,2010-10-02,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0.0
3518,ID_AE3U4I4,2010-01-16,2011-01-15,Female,-5939,2010-01-16,2,NaN,As Attached,TOYOTA,Victoria Island,Lagos,Car Classic,1.0
3649,ID_AUPXOA1,2010-08-04,2011-08-03,Male,-22,2010-08-04,1,Saloon,NaN,Mercedes,Oshodi-Isolo,Oshodi-Isolo,CarSafe,0.0
5173,ID_FKJFVMX,2010-12-14,2011-01-13,Female,-5939,2010-12-14,2,NaN,NaN,NaN,Victoria Island,Lagos,Customized Motor,1.0
6387,ID_J0OJ082,2010-04-23,2011-04-22,Male,-12,2010-04-23,3,Saloon,NaN,Hyundai,Victoria Island,Lagos,Car Plus,0.0
6856,ID_KK04G4Y,2010-04-30,2011-04-29,Female,-76,2010-04-30,2,NaN,NaN,Hyundai,Ilupeju,Lagos,CVTP,0.0


In [121]:
#making the values of the age to be absolute
all_data['Age'] = all_data['Age'].abs()

In [122]:
all_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,13281.0,45.148106,91.859380,0.0,35.0,41.0,50.0,6099.0
No_Pol,13281.0,1.302763,0.723195,1.0,1.0,1.0,1.0,10.0
target,12079.0,0.120457,0.325509,0.0,0.0,0.0,0.0,1.0


In [123]:
all_data[all_data['Age']>144]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
2487,ID_7EU42FJ,2010-01-29,2011-01-28,Male,320,2010-01-29,2,NaN,NaN,TOYOTA,Abuja,Abuja,Car Classic,0.0
3427,ID_A3YN75Z,2010-10-02,2011-10-01,NOT STATED,6099,2010-10-02,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0.0
3493,ID_ABN28TN,2010-09-06,2011-09-05,Male,320,2010-09-06,1,NaN,NaN,Range Rover,Ketu,Lagos,Car Classic,0.0
3518,ID_AE3U4I4,2010-01-16,2011-01-15,Female,5939,2010-01-16,2,NaN,As Attached,TOYOTA,Victoria Island,Lagos,Car Classic,1.0
4511,ID_DJ2OITJ,2010-09-06,2011-09-05,Male,320,2010-09-06,1,NaN,As Attached,Range Rover,Ketu,Lagos,Car Classic,0.0
4969,ID_EXEIEJM,2010-03-25,2011-03-24,Male,320,2010-03-25,2,JEEP,Silver,Range Rover,Victoria Island,Lagos,Car Classic,0.0
4975,ID_EY7TG7T,2010-11-26,2011-11-25,Male,320,2010-11-26,1,NaN,NaN,NaN,Ketu,Lagos,Car Classic,0.0
5173,ID_FKJFVMX,2010-12-14,2011-01-13,Female,5939,2010-12-14,2,NaN,NaN,NaN,Victoria Island,Lagos,Customized Motor,1.0
5730,ID_H6FN9MZ,2010-01-22,2011-01-21,Male,320,2010-01-22,2,NaN,NaN,Honda,Lagos Mainland,Lagos,Car Classic,0.0
5885,ID_HNBGWO8,2010-09-01,2011-08-31,Male,320,2010-09-01,1,NaN,NaN,TOYOTA,Festac,Benue,Car Classic,0.0


In [124]:
#It seems a common error on 320, 5939, 6099 years
#we replace 320 to 32, 5939 to 59, 6099 to 60
all_data['Age'].replace({320:32, 5939:59, 6099:60}, inplace=True)

In [125]:
#Rechecking the range for the age
all_data['Age'].describe().transpose()

count    13281.000000
mean        43.547700
std         19.397184
min          0.000000
25%         35.000000
50%         41.000000
75%         50.000000
max        144.000000
Name: Age, dtype: float64

## **3.2. Dealing With LGA_Name and States**

In [126]:
all_data[(all_data.LGA_Name.isnull()) & (~all_data.State.isnull())]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
9380,ID_S1O477F,2010-12-20,2011-12-19,Male,38,2010-12-20,1,NaN,Black,TOYOTA,NaN,N-A,Car Classic,0.0


In [127]:
all_data.loc[all_data['State']=='N-A','State'] = np.NaN

In [128]:
all_data[(~all_data.LGA_Name.isnull()) & (all_data.State.isnull())]

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
297,ID_0X9FT1B,2010-08-02,2011-01-23,Female,41,2010-08-02,1,Saloon,Black,TOYOTA,LGA,NaN,Car Classic,0.0
568,ID_1S28LSA,2010-08-01,2011-07-31,Male,30,2010-08-01,1,Saloon,White,Honda,LGA,NaN,Car Classic,0.0
1024,ID_3699IFV,2010-01-19,2011-01-18,Joint Gender,35,2010-01-19,1,NaN,Silver,Lexus,LGA,NaN,Car Plus,0.0
1168,ID_3LERA4V,2010-11-18,2011-11-17,Female,35,2010-11-18,1,NaN,NaN,NaN,LGA,NaN,Car Classic,0.0
1262,ID_3UVSEFO,2010-11-22,2011-11-21,Female,40,2010-11-22,2,NaN,NaN,NaN,LGA,NaN,Car Plus,0.0
2544,ID_7L9SS9R,2010-07-31,2011-07-30,Female,42,2010-07-31,1,Saloon,Grey,TOYOTA,LGA,NaN,Car Classic,1.0
3583,ID_AO08COK,2010-04-12,2011-04-11,Male,52,2010-04-12,1,JEEP,Green,TOYOTA,LGA,NaN,Car Classic,0.0
4717,ID_E63UTB8,2010-07-31,2011-07-30,Female,42,2010-07-31,1,Saloon,Grey,TOYOTA,LGA,NaN,Car Classic,0.0
7321,ID_M0XED7B,2010-12-20,2011-12-19,Female,42,2010-12-20,1,Saloon,NaN,TOYOTA,LGA,NaN,CarSafe,0.0
7608,ID_MZK0QIU,2010-01-04,2011-01-03,Male,51,2010-01-04,1,JEEP,NaN,Honda,LGA,NaN,CarSafe,0.0


In [129]:
all_data.loc[all_data['LGA_Name']=='LGA','LGA_Name'] = np.NaN

In [130]:
StateName.head()

,LGA,State
0,Abadam,Borno State
1,Abaji,Federal Capital Territory
2,Abak,Akwa Ibom State
3,Abakaliki,Ebonyi State
4,Aba-North,Abia State


In [131]:
StateName['LGA'] = StateName.LGA.str.replace('--',' ').str.replace('-',' ').str.title()


In [132]:
all_data['LGA_Name'] = all_data.LGA_Name.str.replace('-',' ').str.replace('/',' ').str.title()
all_data['LGA_Name'] = all_data.LGA_Name.str.replace('.',' ').str.replace(',','').str.strip()


In [133]:
LGA_not_found = []
for lga_name in all_data.LGA_Name:
  if lga_name not in StateName.LGA.unique():
    LGA_not_found.append(lga_name)

len(list(set(LGA_not_found)))

14

In [134]:
list(set(LGA_not_found))

[nan,
 'AjegunleLagos  State',
 'Ilesha West',
 'Ogba',
 'Ifako Ijaye',
 'Calabar Municipality',
 'Ovia Southwest',
 'Ilesha East',
 'Olamabolo',
 'Ogbmosho South',
 'Obia Akpor',
 'Somolu',
 'Bekwara',
 'Ogudu']

In [135]:
for lga in StateName.LGA.unique():
  if  (type(lga) != float):
    if ('Shomolu' in lga):
      print(lga)

Shomolu


In [136]:
correct_LGA = {
               'Obia Akpor':'Obio Akpor',
               'Ogbmosho South':'Ogbomosho South',
               'Ovia Southwest':'Ovia South West',
               'Ovia Southwest':'Ovia South West',
               'Ilesha East':'Ilesa East',
               'Ilesha West':'Ilesa West',
               'Ogudu':'Ojota',
               'Olamabolo':'Olamaboro',
                'Calabar Municipality':'Calabar Municipal',
               'Calabar Municipality':'Calabar Municipal',
               'Ifako Ijaye':'Ifako Ijaiye',
               'AjegunleLagos  State':'Ajegunle Lagos State',
               'Bekwara':'Bekwarra',
               'Ogba':'Ogbadibo',
               'Somolu':'Shomolu'
               }

all_data['LGA_Name'] = all_data['LGA_Name'].replace(correct_LGA)

In [137]:
all_data['State'] = all_data['LGA_Name'].replace(dict(zip(StateName.LGA,StateName.State)))

##**3.3. Dealing With Subject_Car_Colour**

In [138]:
all_data.Subject_Car_Colour.value_counts().to_frame()

,Subject_Car_Colour
Black,2057
Silver,605
Grey,565
As Attached,555
Blue,398
White,321
Red,274
Green,259
Gold,192
Ash,142


In [139]:
modified_colors = {'Ash':'Grey',
                   'Wine':'Red',
                   'Cream':'White',
                   'Champagne':'Beige',
                   'Burgundy':'Red'}

all_data['Subject_Car_Colour'] = all_data.Subject_Car_Colour.replace(modified_colors)

In [140]:
Colors = ['Black','Silver','As Attached','Blue','Red','White',
          'Green','Gold','Brown','Yellow','Orange','Purple','Beige']
        
def which_color(x,color):
  if type(x) != float:  
    if color in x:
      return 1
    else:
      return 0

for Color in Colors:
  all_data['Car_Color_'+Color] = all_data['Subject_Car_Colour'].apply(lambda x: which_color(x,Color))

def Gray_Or_Grey(x):
  if type(x) != float:
    if ('Gray' in x) or ('Grey' in x):
      return 1
    else:
      return 0

all_data['Car_Color_Gray'] = all_data['Subject_Car_Colour'].apply(lambda x: Gray_Or_Grey(x))

In [141]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      13281 non-null  object 
 1   Policy Start Date       13281 non-null  object 
 2   Policy End Date         13281 non-null  object 
 3   Gender                  12881 non-null  object 
 4   Age                     13281 non-null  int64  
 5   First Transaction Date  13281 non-null  object 
 6   No_Pol                  13281 non-null  int64  
 7   Car_Category            9171 non-null   object 
 8   Subject_Car_Colour      5622 non-null   object 
 9   Subject_Car_Make        10557 non-null  object 
 10  LGA_Name                6136 non-null   object 
 11  State                   13281 non-null  object 
 12  ProductName             13281 non-null  object 
 13  target                  12079 non-null  float64
 14  Car_Color_Black         5622 non-null 

In [142]:
all_data['Car_Category'] = all_data['Car_Category'].str.replace(' ','-').str.lower()

In [143]:
small = ['motorcycle', 'shape-of-vehicle-chasis'],
medium = ['saloon', 'sedan', 'camry-car-hire'],
large = ['truck', 'pick-up', 'van', 'mini-bus', 'wagon', 'mini-van', 'jeep'],
extra_large = ['pick-up->-3-tons', 'tipper-truck', 'station-4-wheel', 'bus']

def car_size(x):
    if x in small:
        return 'small'
    if x in medium:
        return 'medium'
    if x in large:
        return 'large'
    if x in extra_large:
        return 'extra_large'

all_data['Car_Category'] = all_data['Car_Category'].apply(car_size)

In [144]:
all_data.drop(['First Transaction Date'],axis=1,inplace=True)

In [145]:
all_data['Gender'].value_counts()

Male            8356
Female          3679
Entity           300
Joint Gender     238
NOT STATED       190
NO GENDER         76
SEX               42
Name: Gender, dtype: int64

In [146]:
all_data['Gender'] = all_data['Gender'].replace({'Entity': 'Other',
                                                'Joint Gender':'Other',
                                                'NOT STATED':'Other',
                                                'NO GENDER': 'Other',
                                                'SEX':'Other'})
all_data['Gender'].value_counts()

Male      8356
Female    3679
Other      846
Name: Gender, dtype: int64

In [147]:
date_cols = [col for col in all_data.columns if 'Date' in col]
num_cols = ['Age','No_Pol']
cat_cols = [col for col in all_data.columns if col not in date_cols+num_cols+['ID', 'target']]

for col in all_data.columns:
  if col in date_cols:
    all_data[col] = pd.to_datetime(all_data[col])
  elif col in cat_cols:
    all_data[col] = all_data[col].astype('category')

# Confirm whether the changes have been applied successfully
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     13281 non-null  object        
 1   Policy Start Date      13281 non-null  datetime64[ns]
 2   Policy End Date        13281 non-null  datetime64[ns]
 3   Gender                 12881 non-null  category      
 4   Age                    13281 non-null  int64         
 5   No_Pol                 13281 non-null  int64         
 6   Car_Category           65 non-null     category      
 7   Subject_Car_Colour     5622 non-null   category      
 8   Subject_Car_Make       10557 non-null  category      
 9   LGA_Name               6136 non-null   category      
 10  State                  13281 non-null  category      
 11  ProductName            13281 non-null  category      
 12  target                 12079 non-null  float64       
 13  C

In [148]:
# Fill in missing values
# For cat cols and date cols fill in with mode and for num cols fill in with 9999
for col in all_data.columns:
  if col in date_cols+cat_cols:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])
  elif col in num_cols:
    all_data[col] = all_data[col].fillna(all_data[col].fillna(round(all_data[col].mean())))

# Confirm that there aren't any missing values
all_data[all_data.columns.difference(['target'])].isna().sum()

Age                      0
Car_Category             0
Car_Color_As Attached    0
Car_Color_Beige          0
Car_Color_Black          0
Car_Color_Blue           0
Car_Color_Brown          0
Car_Color_Gold           0
Car_Color_Gray           0
Car_Color_Green          0
Car_Color_Orange         0
Car_Color_Purple         0
Car_Color_Red            0
Car_Color_Silver         0
Car_Color_White          0
Car_Color_Yellow         0
Gender                   0
ID                       0
LGA_Name                 0
No_Pol                   0
Policy End Date          0
Policy Start Date        0
ProductName              0
State                    0
Subject_Car_Colour       0
Subject_Car_Make         0
dtype: int64

In [149]:
for col in date_cols:
  for date_feature in ['day','dayofweek','dayofyear','year', 'month','quarter','weekday','weekofyear']:
    all_data[col+' '+date_feature] = getattr(all_data[col].dt, date_feature)
    # cat_cols.append(col+' '+date_feature) 
    
all_data.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target,Car_Color_Black,Car_Color_Silver,Car_Color_As Attached,Car_Color_Blue,Car_Color_Red,Car_Color_White,Car_Color_Green,Car_Color_Gold,Car_Color_Brown,Car_Color_Yellow,Car_Color_Orange,Car_Color_Purple,Car_Color_Beige,Car_Color_Gray,Policy Start Date day,Policy Start Date dayofweek,Policy Start Date dayofyear,Policy Start Date year,Policy Start Date month,Policy Start Date quarter,Policy Start Date weekday,Policy Start Date weekofyear,Policy End Date day,Policy End Date dayofweek,Policy End Date dayofyear,Policy End Date year,Policy End Date month,Policy End Date quarter,Policy End Date weekday,Policy End Date weekofyear
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,1,extra_large,Black,TOYOTA,Victoria Island,Unknown,Car Classic,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,4,134,2010,5,2,4,19,13,4,133,2011,5,2,4,19
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,1,extra_large,Grey,TOYOTA,Victoria Island,Unknown,Car Classic,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29,0,333,2010,11,4,0,48,28,0,332,2011,11,4,0,48
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,1,extra_large,Red,TOYOTA,Victoria Island,Unknown,Car Classic,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,6,80,2010,3,1,6,11,20,6,79,2011,3,1,6,11
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,1,extra_large,Black,TOYOTA,Victoria Island,Unknown,CarSafe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,5,233,2010,8,3,5,33,20,5,232,2011,8,3,5,33
4,ID_00BRP63,2010-08-29,2010-12-31,Other,20,3,extra_large,Black,TOYOTA,Lagos,State-Lagos,Muuve,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,6,241,2010,8,3,6,34,31,4,365,2010,12,4,4,52


In [150]:
# ss = [x for x in all_data.columns if x not in ['ID', 'target']+date_cols]
# all_data['sum'] = all_data[ss].sum(axis=1)  
# all_data['min'] = all_data[ss].min(axis=1)
# all_data['max'] = all_data[ss].max(axis=1)
# all_data['mean'] = all_data[ss].mean(axis=1)
# all_data['std'] = all_data[ss].std(axis=1)
# all_data['skew'] = all_data[ss].skew(axis=1)
# all_data['kurt'] = all_data[ss].kurtosis(axis=1)
# all_data['med'] = all_data[ss].median(axis=1)
# all_data['range'] = all_data['max']-all_data['min']

In [151]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in cat_cols:
  all_data[col] = encoder.fit_transform(all_data[col])

In [152]:
# for col in cat_cols:
#   all_data[col] = all_data[col].map(train_df.groupby(col)['target'].mean())

# **4. Buinding Model**

In [153]:
# Separate train and test data from the combined dataframe
train_df = all_data[:ntrain]
test_df = all_data[ntrain:]

# Check the shapes of the split dataset
train_df.shape, test_df.shape

((12079, 43), (1202, 43))

In [154]:
# Select main columns to be used in training
features = train_df.columns.difference(['ID','target','Policy End Date','Policy Start Date',
                                       'Policy End Date month','Policy Start Date month','Policy End Date year'])

In [155]:

X = train_df[features]
y = train_df.target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=123)

In [156]:
train_df[train_df['target']==0].shape[0]/train_df.shape[0]

0.8795430085271959

In [157]:
# lgbm = LGBMClassifier(n_estimators=800,
#                        num_leaves=20,
#                        min_child_samples=11,
#                        class_weight={0:0.2, 1:0.8})

# rf = RandomForestClassifier(n_estimators=100,
#                             max_depth=16,
#                             class_weight={0:0.14,1:0.86},
#                             random_state=0)

# xgb = XGBClassifier(n_estimators=1000,
#                     max_depth=8,
#                     scale_pos_weight=7)

# model = VotingClassifier(estimators=[('lgbm',lgbm),('rf',rf),('xgb',xgb)])


In [268]:
w = 0.12

model = LGBMClassifier(n_estimators=800,
                       num_leaves=20,
                       min_child_samples=11,
                       class_weight={0:w,1:1-w})                 

# model = RandomForestClassifier(n_estimators=100,
#                                max_depth=6,
#                                class_weight={0:w,1:1-w},
#                                random_state=0)

# Create a StratifiedKFold object
str_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

fold_metrics = []
n_fold = 1

for train_index, test_index in str_kf.split(X_train,y_train):
  cv_train, cv_test = X.iloc[train_index], X.iloc[test_index]
  y_train_cv, y_test_cv = y[train_index], y[test_index]

# Train a model
  model.fit(cv_train,y_train_cv)
# Make predictions
  predictions = model.predict(cv_test)
# Calculate the metric
  metric = f1_score(y_test_cv, predictions)
  fold_metrics.append(metric)
  print(f'fold {n_fold}: Done')
  n_fold += 1

np.mean(fold_metrics), np.std(fold_metrics)

fold 1: Done
fold 2: Done
fold 3: Done
fold 4: Done
fold 5: Done


(0.34687669147312306, 0.025719826360184737)

In [269]:
0.35102951443374425

0.35102951443374425

In [270]:
Local_val = round(np.mean(fold_metrics),4)

In [271]:
f1_score(y_test,model.predict(X_test))

0.7093023255813953

In [272]:
model.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight={0: 0.12, 1: 0.88},
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=11, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=800, n_jobs=-1, num_leaves=20,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [273]:
# ax=lgbm.plot_importance(model, figsize=(15,10))
# plt.show()

In [274]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 12079 to 13280
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   Age                           1202 non-null   int64
 1   Car_Category                  1202 non-null   int64
 2   Car_Color_As Attached         1202 non-null   int64
 3   Car_Color_Beige               1202 non-null   int64
 4   Car_Color_Black               1202 non-null   int64
 5   Car_Color_Blue                1202 non-null   int64
 6   Car_Color_Brown               1202 non-null   int64
 7   Car_Color_Gold                1202 non-null   int64
 8   Car_Color_Gray                1202 non-null   int64
 9   Car_Color_Green               1202 non-null   int64
 10  Car_Color_Orange              1202 non-null   int64
 11  Car_Color_Purple              1202 non-null   int64
 12  Car_Color_Red                 1202 non-null   int64
 13  Car_Color_Silver            

In [275]:
# Make prediction on the test set
test_df = test_df[features]
  
predictions = model.predict(test_df)

# Create a submission file
sub_file = SS.copy()
sub_file.target = predictions

In [276]:
# Create a csv file and upload to zindi 
sub_file.to_csv(f'IndabaX_Ghana_Sub({Local_val}).csv', index = False)